In [ ]:
# coding: utf8

#Import

In [ ]:
import os
import sys
import json
import pprint
from pyes import *
import pandas as pd
pp = pprint.PrettyPrinter(indent=4)
from datetime import datetime, date

#Confinguration

In [ ]:
HERE = os.getcwd() + '/'
PROJECTS_CSV = u'projects.csv' 

#Class project

In [ ]:
class project():
    """
    """
    
    def __init__(self, name, desc='...', date=date.today(), link='#', icon='', team=''):
        self.name = name
        self.description = desc
        self.date = date
        self.link = link
        self.icon = icon
        self.team = team
        
        self.coworkers = []
        self.technologies = []
        
        self._id = 0
        
    def setId(self, _id):
        self._id = _id
        
    @property
    def getId(self):
        return self._id
    
    def addCoworkers(self, coworkers=[]):
        for f in coworkers: 
            if f not in self.coworkers: self.coworkers.append(f)

    def addTech(self, tech=[]):
        for t in tech: 
            if t not in self.technologies: self.technologies.append(t)
    
    @property
    def show(self):
        return {u'name': self.name,
           u'description': self.description,
           u'date': self.date.strftime(u'%Y-%m-%d'),
           u'link': self.link,
           u'icon': self.icon,
           u'team': self.team,
           u'coworkers': self.coworkers,
           u'technologies': self.technologies
          }

#Loader

### connection

In [ ]:
esConn = ES('127.0.0.1:9200')

In [ ]:
#add mapping
mapping = {}

In [ ]:
df = pd.read_csv(HERE + PROJECTS_CSV, sep=';', lineterminator=u'\r')

In [ ]:
df.head()

In [ ]:
for idx, p in df.iterrows():
    pro = project(name = p[u'name'],
                  desc = u'...', 
                  date = datetime.strptime(p[u'date'], u'%d/%m/%y'),
                  link = p[u'link'],
                  icon = p[u'icon'], team = p[u'team'])
    pro.addCoworkers(p[u'coworkers'].split('|'))
    pro.addTech(p[u'technologies'].split('|'))
    response = esConn.index(pro.show, 'lmd', 'project', p[u'id'])
    pp.pprint(response)
    #pp.pprint(pro.show)

In [ ]:
lmd = project(name=u'This web site is also a project (',
                  desc=u'...', 
                  date=date(2015, 4, 1),
                  link=u'',
                  icon=u'dd.svg', team=u'Me')

lmd.addTech([u'Ember.js', u'Skeleton.css', u'Divshot'])

In [ ]:
esConn.index(lmd.show, 'lmd', 'project')

In [ ]:
#curl -XDELETE http://localhost:9200/lmd/project/?pretty
esConn.delete('lmd', 'project', '', bulk=False)

In [ ]:
#esConn.get('lmd', 'project', None)